Structured variational message passing demo
===

A generalization of VMP under the mean field approximation is structured VMP (SVMP). SVMP allows arbitrary factorizations of the approximating $q$ distribution (Dauwels 2007). As an example case to illustrate and test our SVMP implementation in ForneyLab, we estimate the mean $m$ and precision $\gamma$ for a set of samples $y = \{y_1, ..., y_k, ..., y_n\}$ drawn from a Gaussian distribution.

The factor graph below shows our generative model, where we use the notation of (Reller, 2012; State-space methods in statistical signal processing). Filled black nodes represent observed variables and dotted arcs denote the repetition of a section.

<img src="images/gauss_est_prec.png">

This is the same model as in the previous demo, however this time we use precision parametrization and factorize the approximating distribution as $q(m,\gamma,y)=q(m,\gamma)\,q(y)$. Also, we estimate the mean and precision in a sample-by sample (online) fashion instead of in bulk.

We begin by implementing the factor graph in ForneyLab and define the structured factorization.

In [1]:
using(ForneyLab)

# Initialize nodes and edges
GaussianNode(id=:g, form=:precision)
EqualityNode(id=:m_eq) # Equality node for mean
EqualityNode(id=:gam_eq) # Equality node for precision
TerminalNode(Delta(), id=:y) # Observed y values are stored in y_node
PriorNode(Gaussian(m=0.0, V=100.0), id=:m_n_min_1) # Prior
PriorNode(Gamma(a=1.0, b=0.01), id=:gam_n_min_1) # Prior
TerminalNode(vague(Gaussian), id=:m_n)
TerminalNode(vague(Gamma), id=:gam_n)

Edge(n(:g).i[:out], n(:y), id=:y)
Edge(n(:m_eq).i[3], n(:g).i[:mean], id=:m)
Edge(n(:gam_eq).i[3], n(:g).i[:precision], id=:gam)
Edge(n(:m_n_min_1), n(:m_eq).i[1])
Edge(n(:gam_n_min_1), n(:gam_eq).i[1])
Edge(n(:m_eq).i[2], n(:m_n))
Edge(n(:gam_eq).i[2], n(:gam_n))

# Set wraps
Wrap(n(:m_n), n(:m_n_min_1))
Wrap(n(:gam_n), n(:gam_n_min_1));

Now we generate some data and define the read and write buffers.

In [2]:
# Generate some data
true_mean = 5.0
true_prec = 0.5
N         = 100 # Draw 100 samples
n_its     = 5 # Iterations per sample
y_data = randn(N)*(1/sqrt(true_prec)) + true_mean

# Add read and write buffers
attachReadBuffer(n(:y), deepcopy(y_data))

m_est = attachWriteBuffer(n(:m_eq).i[2])
gam_est = attachWriteBuffer(n(:gam_eq).i[2]);

We define the structured factorization by grouping edges that belong to the same factor. 

In [3]:
# Define the structured variational algorithm
factor([eg(:m), eg(:gam)])
factor(eg(:y))
initialize([eg(:m), eg(:gam)], vague(NormalGamma))
initialize(eg(:y), vague(Gaussian))

algo = VariationalBayes(n_iterations=n_its)

VariationalBayes inference algorithm
    number of subgraphs: 2
    number of iterations: 5


In [4]:
# Inspect the structured factorization
draw(algo.recognition_factorization.subgraphs[1])
draw(algo.recognition_factorization.subgraphs[2])

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.36.0 (20140111.2315)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 8520972782901914155 -->
 8520972782901914155 
 
 m_n 
 
<!-- 12365311401188345756 -->
 12365311401188345756 
 
 gam_n_min_1 
 
<!-- 12956371063457593399 -->
 12956371063457593399 
 
 = 
 gam_eq 
 
<!-- 12365311401188345756->12956371063457593399 -->
 12365311401188345756->12956371063457593399 
 
 
 1 1 
 1 out 
 
<!-- 11562856502394274663 -->
 11562856502394274663 
 
 gam_n 
 
<!-- 12956371063457593399->11562856502394274663 -->
 12956371063457593399->11562856502394274663 
 
 
 1 out 
 2 2 
 
<!-- 17268050207938340035 -->
 17268050207938340035 
 
 N 
 g 
 
<!-- 12956371063457593399->17268050207938340035 -->
 12956371063457593399->17268050207938340035 
 
 
 2 precision 
 3 3 
 
<!-- 12087315154755581506 -->
 12087315154755581506 
 
 = 
 m_eq 
 
<!-- 12087315154755581506->8520972782901914155 -->
 12087315154755581506->8520972782901914155 
 
 
 1 out 
 2 2 
 
<!-- 12087315154755581506->17268050207938340035 -->
 12087315154755581506->17268050207938340035 
 
 
 1 mean 
 3 3 
 
<!-- 7834114888913362232 -->
 7834114888913362232 
 
<!-- 17268050207938340035->7834114888913362232 -->
 17268050207938340035->7834114888913362232 
 
 
 1 out 
 3 out 
 
<!-- 12564404526568826281 -->
 12564404526568826281 
 
 m_n_min_1 
 
<!-- 12564404526568826281->12087315154755581506 -->
 12564404526568826281->12087315154755581506 
 
 
 1 1 
 1 out

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.36.0 (20140111.2315)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 7834114888913362232 -->
 7834114888913362232 
 
 y 
 
<!-- 17268050207938340035 -->
 17268050207938340035 
 
 N 
 g 
 
<!-- 17268050207938340035->7834114888913362232 -->
 17268050207938340035->7834114888913362232 
 
 
 1 out 
 3 out 
 
<!-- 12956371063457593399 -->
 12956371063457593399 
 
<!-- 12956371063457593399->17268050207938340035 -->
 12956371063457593399->17268050207938340035 
 
 
 2 precision 
 3 3 
 
<!-- 12087315154755581506 -->
 12087315154755581506 
 
<!-- 12087315154755581506->17268050207938340035 -->
 12087315154755581506->17268050207938340035 
 
 
 1 mean 
 3 3

In [5]:
run(algo);

Let's inspect the results.

In [6]:
println("True mean: $(true_mean)")
println("True precision: $(true_prec)")
println("Number of samples: $(N)")
println("Sample mean: $(round(mean(y_data),2))")
println("Sample precision: $(round(1/var(y_data),2))")
println("\n----- Online estimation after $(n_its) VMP updates per sample -----")
println("Mean estimate: $(round(mean(m_est[end])[1],2)), with variance $(round(var(m_est[end])[1,1],2))")
println("Precision estimate: $(round(mean(gam_est[end]),2)), with variance $(round(var(gam_est[end]),2))")

True mean: 5.0
True precision: 0.5
Number of samples: 100
Sample mean: 4.92
Sample precision: 0.42

----- Online estimation after 5 VMP updates per sample -----
Mean estimate: 4.91, with variance 0.05
Precision estimate: 0.3, with variance 0.0
